In [1]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2Processor
import torch
import os
import librosa
import IPython

2025-07-22 13:17:54.437623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753190274.471684    1140 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753190274.482060    1140 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def load_wav(path, sr):
    return librosa.load(path, sr = sr)[0]

In [ ]:
PRETRAINED_PATH = "facebook/wav2vec2-base"
SAVED_DIR = './saved'
SPECIAL_TOKENS={
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "unk_token": "<unk>",
    "pad_token": "<pad>"
}


tokenizer = Wav2Vec2CTCTokenizer("vocab.json", **SPECIAL_TOKENS, word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(PRETRAINED_PATH)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

model = Wav2Vec2ForCTC.from_pretrained(
    PRETRAINED_PATH,
    ctc_loss_reduction="sum",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    gradient_checkpointing=False
)

/opt/conda/lib/python3.12/site-packages/transformers/configuration_utils.py:309: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
checkpoint_path = os.path.join(SAVED_DIR, "best_model.tar")
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [5]:
waveform = load_wav('./yeunhau.wav', sr=16000)
transcript = 'Khi hôn, các cô gái sẽ có những phản ứng sinh lý như vô thức ôm chặt cơ thể bạn, cảm giác như họ gọt sát vào bạn. Hai người sẽ sát lại gần nhau, sau đó nhịp tim bắt đầu nhanh hơn. Ở đầu dên à lần cũng sẽ tăng lên, khiến toàn thân nóng bừng, chân tay trở nên mềm nhũn. Nếu nụ hôn đủ nồng nhiệt, cô ấy sẽ cảm thấy cơ thể yếu dần, đầu óc như trống rỗng. Cả người sẽ mất thăng bằng và ngã vào người bạn. Sau khi hôn xong, mặt cô ấy sẽ đỏ bừng, cúi đầu không dám nhìn bạn nữa.'
print(transcript)
IPython.display.Audio(waveform, rate=16000)

Khi hôn, các cô gái sẽ có những phản ứng sinh lý như vô thức ôm chặt cơ thể bạn, cảm giác như họ gọt sát vào bạn. Hai người sẽ sát lại gần nhau, sau đó nhịp tim bắt đầu nhanh hơn. Ở đầu dên à lần cũng sẽ tăng lên, khiến toàn thân nóng bừng, chân tay trở nên mềm nhũn. Nếu nụ hôn đủ nồng nhiệt, cô ấy sẽ cảm thấy cơ thể yếu dần, đầu óc như trống rỗng. Cả người sẽ mất thăng bằng và ngã vào người bạn. Sau khi hôn xong, mặt cô ấy sẽ đỏ bừng, cúi đầu không dám nhìn bạn nữa.


In [6]:
def clean_text(text: str, chars_to_ignore: str = r'[,?.!\-;:"“%\'�]') -> str:
    """
    Clean a transcript string by removing special characters and lowering the case.

    Args:
        text (str): The input transcript.
        chars_to_ignore (str): Regex pattern of characters to remove.

    Returns:
        str: Cleaned transcript.
    """
    cleaned = re.sub(chars_to_ignore, '', text).lower()
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

In [7]:
import torch
import re
from jiwer import wer

model.eval()
inputs = processor(waveform.squeeze(), sampling_rate=16000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(**inputs).logits

predicted_ids = torch.argmax(logits, dim=-1)
results = processor.batch_decode(predicted_ids)
cleaned_results = []
for text in results:
    text = re.sub(r'<unk>|</s>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    cleaned_results.append(text)
    
reference = clean_text(transcript)
hypothesis = clean_text(cleaned_results[0])

error = wer(reference, hypothesis)
print(f"Expected: [{reference}]\n")
print(f"Actual:   [{hypothesis}]\n")
print(f"WER: {error:.4f}")


Expected: [khi hôn các cô gái sẽ có những phản ứng sinh lý như vô thức ôm chặt cơ thể bạn cảm giác như họ gọt sát vào bạn hai người sẽ sát lại gần nhau sau đó nhịp tim bắt đầu nhanh hơn ở đầu dên à lần cũng sẽ tăng lên khiến toàn thân nóng bừng chân tay trở nên mềm nhũn nếu nụ hôn đủ nồng nhiệt cô ấy sẽ cảm thấy cơ thể yếu dần đầu óc như trống rỗng cả người sẽ mất thăng bằng và ngã vào người bạn sau khi hôn xong mặt cô ấy sẽ đỏ bừng cúi đầu không dám nhìn bạn nữa]

Actual:   [khi hon cac cô gài sẽ cò những phản ứng xinh lý nào trước viên họ sẽ vô thước ông trật cơ thể bá càn ráp như họ cọ sát vào bạn hai người sẽ sác lại gần nhau sau đó hi pim bắt đầu nhanh hơn ở đou zenelon cũng sẽ tăng le yến toàn thân nóng vường chân tay trở nên mẻn nú nếu nỗ hon đủ nông nghiệt cô ồi sẽ cảm thấy cơ thể yếu dân đầu óc như chồng rống cà người sẽ mất thâng bằm kã vào người bạn sau khi hôn song mà sẽ đỏo bừng cuối đầu khủng giá mình bạn nữ]

WER: 0.4685


In [ ]:
def compute_wer(self, logits, labels):
        pred_ids = torch.argmax(logits, dim=-1)
        pred_str = self.processor.batch_decode(pred_ids, skip_special_tokens=True)
        label_str = self.processor.batch_decode(labels, skip_special_tokens=True)
        return wer(label_str, pred_str)

In [8]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total:,}")
    print(f"Trainable parameters: {trainable:,}")

count_parameters(model)


Total parameters: 94,447,843
Trainable parameters: 94,447,843
